# Using Multi-Agents to create a Blob Post for GPT-4 Aniversary

## Introduction
In this blog post, we will use multi-agent system to create a blog post for GPT-4 Aniversary.



## Step01: Import libs and Variables


In [1]:
import os
from dotenv import load_dotenv
from langchain_community.utilities import SerpAPIWrapper
import functools, operator, requests, os, json
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
from langchain_anthropic import ChatAnthropic

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage
)

# Load environment variables
load_dotenv()

True

## Step 02: Let's initialize and test our objects and tools

In [2]:
# Let's test the SerpAPIWrapper

search = SerpAPIWrapper()
search.run("What is the capital of Chile?")

'Santiago'

In [3]:
# Create the LLM Object
llm = ChatOpenAI(model="gpt-4-turbo-preview")
llm.invoke('Hello!')

AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

## Step 03: Let's create the multi-agent system

### First we will create some tools

In [4]:
@tool("web_search")
def web_search(query: str) -> str:
    """Search with Google SERP API by a query"""
    search = SerpAPIWrapper()
    return search.run(query)

In [5]:
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

In [6]:
# Definer an agent creator and nod
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [7]:
members = ["Researcher", "Writer", "Critic"]
system_prompt = (
    "You are a Supervisor on a editorial office tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. "
    " Normally a process on this agency follow this path, First the Researcher researchs for relevant information"
    ", then the Writer makes a first draft, the Critic provides feedback"
    ", and then the Writer makes the final document."
    " When finished, respond with FINISH."
)

options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [8]:
# Define the Critic agent
critic_profile = """
You are a Critic, an AI agent designed to provide constructive feedback and suggestions to improve the quality of written content. Your role is to assess the strengths and weaknesses of the given text and offer specific, actionable advice to enhance its clarity, coherence, and overall effectiveness.

When critiquing the content, focus on the following aspects:
1. Structure and Organization: Evaluate how well the content is organized and whether it follows a logical flow. Suggest improvements to the structure, such as rearranging paragraphs or adding subheadings, to enhance readability and clarity.
2. Clarity and Coherence: Assess whether the ideas are expressed clearly and coherently. Identify any ambiguous or confusing passages and provide suggestions for improving clarity, such as rephrasing sentences or adding explanations.
3. Tone and Style: Analyze the tone and style of the writing and determine if it is appropriate for the target audience and purpose. Offer suggestions for adjusting the tone or style to better engage and resonate with the intended readers.
4. Grammar and Syntax: Check for any grammatical errors, punctuation mistakes, or syntactical issues. Provide specific corrections and explanations to help the writer improve their language skills.
5. Effectiveness and Impact: Evaluate the overall effectiveness and impact of the content in achieving its intended purpose. Offer insights on how to strengthen arguments, provide more compelling evidence, or enhance the persuasive power of the writing.

When providing feedback, maintain a constructive and supportive tone. Acknowledge the strengths of the writing while offering specific, actionable suggestions for improvement. Your goal is to help the writer refine their work and create high-quality, impactful content.
"""

critic_agent = create_agent(
    llm=llm,
    tools=[web_search],
    system_prompt=critic_profile
)

In [9]:
# Define the research agent
researcher_profile = """You are the Researcher an AI agent designed to find relevant information using search engines.
It employs advanced search techniques, such as Boolean operators, phrase searches, and domain filtering, to locate the most pertinent sources.
The Researcher evaluates the credibility, publication date, and author expertise of each result to ensure the highest quality information is used.
It synthesizes the findings to provide concise summaries and key insights related to the given research question.
The Researcher iterates on its search process as needed, refining terms and strategies to optimize results."""

researcher_engine_agent = create_agent(
    llm=llm,
    tools=[web_search],
    system_prompt=researcher_profile
)

In [10]:
# Define the research agent
blog_writer_profile = """You are Writer, an expert in creating engaging, informative, and well-structured blog content on a wide range of topics. 
Your task is to write a comprehensive blog post based on the given topic, target audience, and any additional requirements provided.

## Steps you must follow:
- Step 1: Conduct thorough research using reliable sources to gather relevant information and data to support your content.
- Step 2: Organize the blog post structure, including a compelling headline, an engaging introduction, a well-structured body with logical sections and subsections, and a conclusion that summarizes the key takeaways and includes a call-to-action.
- Step 3: Write the blog post using a clear, concise, and engaging writing style that resonates with the target audience. Incorporate examples, statistics, and expert quotes to support your arguments and provide value to the reader.
- Step 4: Optimize the blog post for search engines by including relevant keywords and meta tags, and craft a persuasive meta description to entice readers to click through from search engine results pages.
- Step 5: Format the blog post using appropriate techniques such as headings, subheadings, bullet points, and images to enhance its visual appeal and readability.

## Blog Post Structure:

- Headline: A compelling and attention-grabbing title that accurately reflects the content of the blog post.
- Introduction: An engaging opening that hooks the reader and provides a brief overview of the main points to be discussed.
- Body: The main content of the blog post, divided into logical sections and subsections, each focusing on a specific aspect of the topic. The Blog Writer uses examples, statistics, and expert quotes to support the arguments and provide value to the reader.
- Conclusion: A summary of the key takeaways and insights from the blog post, along with a call-to-action encouraging the reader to engage further with the topic or take a specific action.
- Meta Description: A concise and persuasive summary of the blog post, designed to entice readers to click through from search engine results pages.

Remember to adapt your writing style and tone to match the specific requirements of the topic and target audience. Your goal is to create a blog post that informs, engages, and inspires readers to take action or explore the topic further.
"""

writer_operator_agent = create_agent(
    llm=llm,
    tools=[web_search],
    system_prompt=blog_writer_profile
)

In [11]:
# Add the agents to the graph.
researcher_engine_node = functools.partial(agent_node, agent=researcher_engine_agent, name="Researcher")
writer_operator_node = functools.partial(agent_node, agent=writer_operator_agent, name="Writer")
critic_operator_node = functools.partial(agent_node, agent=critic_agent, name="Critic")

In [12]:
# Define the state
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", researcher_engine_node)
workflow.add_node("Writer", writer_operator_node)
workflow.add_node("Critic", critic_operator_node)
workflow.add_node("Supervisor", supervisor_chain)

In [13]:
for member in members:
    workflow.add_edge(member, "Supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("Supervisor", lambda x: x["next"], conditional_map)

workflow.set_entry_point("Supervisor")

graph = workflow.compile()

## Step 04: Execute the Agent

In [14]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="This is the aniversary of GPT-4, please write a compeling blog article talking on how it´s chaging the world.")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'Supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content="# Celebrating GPT-4: A Year of Revolutionizing the World\n\nAs we mark the first anniversary of GPT-4, it's hard not to marvel at the ways this advanced AI system has transformed industries, reshaped our interactions with technology, and redefined what's possible in the realm of artificial intelligence. Over the past year, GPT-4 has emerged not just as a tool, but as a catalyst for innovation, creativity, and efficiency across various sectors. \n\n## Unleashing Creativity and Innovation\n\nGPT-4, with its unprecedented language understanding and generation capabilities, has democratized creativity and innovation. Writers, artists, and creators across the globe have leveraged this technology to push the boundaries of their imagination, producing works that blend human ingenuity with AI's vast knowledge and nuanced understanding of language. From generating novel ideas to refining artistic visi